In [2]:
import numpy as np
import csv

In [2]:
import osmnx as ox
import pandas as pd
poly = ox.gdf_from_place('Baltimore, Maryland, USA', which_result=2)
# Get buildins gdf based on polygon
gdf = ox.create_buildings_gdf(polygon = poly.geometry[0])

Self-intersection at or near point -76.603476837716656 39.296847746533537
Self-intersection at or near point -76.616428799999994 39.291846100000001
Self-intersection at or near point -76.613582164696879 39.298842597631051


In [3]:
with open('CleanDataBase/vacantBaltimore_171117_8000__OSMNX', 'rb') as f:
    reader = csv.reader(f)
    # vacantosmnx : list[int] - list of all vacant houses id
    vacantosmnx = map(int,list(reader)[0])

In [13]:
# add new column 'housetype'
sLength = len(gdf[gdf.columns[0]])
gdf = gdf.assign(
    housetype=pd.Series(np.zeros(len(gdf[gdf.columns[0]]),dtype = int)).values) 

In [14]:
# set vacant house
for i in gdf.index:
    if i in vacantosmnx:
        gdf.loc[i,'housetype'] = 2     

Done with vacant.


In [15]:
gdf_proj = ox.project_gdf(gdf)

In [ ]:
# df.to_csv('example.csv')

In [20]:
count = 0
# set other types of building
for i in gdf.index:
    # if it's not building
    # note that no wall for this
    if gdf['building'][i] != 'yes':
        gdf.loc[i,'housetype'] = -1
    # if no address for the building or the area of buildig > 398
    # option: area > 125 or area < 37
    #elif pd.isnull(gdf['addr:street'][i]) or gdf_proj.area[i] > 398 : 
        #gdf.loc[i,'housetype'] = 3
    count += 1
    if count % 10000==0:
        print count 

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000


In [ ]:
import time

In [54]:
E = []
count = 0
for i in xrange(len(tempgdf.index)):
    start = time.time()
    for j in xrange(i+1,len(tempgdf.index)):
        
        if set(tempgdf['nodes'][tempgdf.index[i]]) & set(tempgdf['nodes'][tempgdf.index[j]]) :
            E.append((tempgdf.index[i],tempgdf.index[j]))
    
    end = time.time()
    print start-end
    count += 1
    if count % 100 ==0:
        print count, i
    

-10.6834378242


KeyboardInterrupt: 

In [ ]:
with open('EdgeOSMNX_Baltimore', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(E)

Get edge take 30 days....

## Space Complexity


Houses : 255433

Vacants : 7593

Occupied : 247840

Compare Houses Set : 247840 x 7593 = 1,881,849,120

> Run out of space

Edges: Can't Compute

> Time Complexity: O($n^2$) = 65,246,017,489
>
> Time Analysis: 10 seconds for each iteration ( 255433 total )
>> Take 15 days total to get edges.....


In [9]:
2*247840

495680

## Total Complexity Analysis

- num of variables

> - Delta Method: $$= |H| + |E| + |O|\times|V| = |H| + |E| + |C|  = O(H+OV+E) = 1,882,104,553 + |E| $$

> - Big M Method: $$= |H| + |E| + |O| = O(H+E) = 255,433 + |E|$$ 

- num of constraint

> - Delta Method: $$= 1 + 3|E| + 3|C| = O(E+OV) = 3|E| + 5,645,547,361$$

> - Big M Method: $$= 1 + 3|E| + 2|O| = O(E+O) = 3|E| + 495,681$$



In [5]:
247840*7593

1881849120

### Maximize the numbers of houses to be demolished

$$ \begin{array}{rll}
    \text{max} & \sum_{i \in H} x_i \\
    \\[1pt] 
    \text{s.t.} & \sum_{i \in V} c_ix_i +  \sum_{(i,j) \in E} (w_ix_j + w_jx_i) -\sum_{(i,j) \in E}(p_{(i,j)} + w_i+w_j )y_{ij} \leq B_T \\ 
    \\
    & \forall (i,j) \in E \\ 
    & \;\;\;\;\;x_i + x_j - y_{ij} \leq 1 \\
    & \;\;\;\;\;-x_i + y_{ij} \leq 0 \\
    & \;\;\;\;\;-x_j + y_{ij} \leq 0 \\
    \\
    & x_i \in \lbrace 0,1 \rbrace \;\; \forall  i \in H\\
    & y_{ij} \in \lbrace 0,1 \rbrace \;\; \forall  (i,j) \in E \\
    \\
    \end{array}
$$

### Minimize the Total Impact

$$ \begin{array}{rll}
    \text{max} & \sum_{(i,j)\in C} w_{(i,j)} ( 1 - x_i - x_j + y_{ij} )  \\
    \\[1pt] 
    \text{s.t.} & \sum_{i \in V} c_ix_i +  \sum_{(i,j) \in E} (w_ix_j + w_jx_i) -\sum_{(i,j) \in E}(p_{(i,j)} + w_i+w_j )y_{ij} \leq B_T \\
    \\
    & \forall (i,j) \in E \uplus C \\
    & \;\;\;\;\;x_i + x_j - y_{ij} \leq 1 \\
    & \;\;\;\;\;-x_i + y_{ij} \leq 0 \\
    & \;\;\;\;\;-x_j + y_{ij} \leq 0 \\
    \\
    & x_i \in \lbrace 0,1 \rbrace \;\; \forall  i \in H\\
    & y_{ij} \in \lbrace 0,1 \rbrace \;\; \forall  (i,j) \in E  \uplus C \\
    \\
    \end{array}
$$

### Maximize the minimum distance between occupied and vacant

$$ \begin{array}{rll}
    \text{max} & \sum_{ i \in O} t_i \\
    \\[1pt] 
    \text{s.t.} & \sum_{i \in V} c_ix_i +  \sum_{(i,j) \in E} (w_ix_j + w_jx_i) -\sum_{(i,j) \in E}(p_{(i,j)} + w_i+w_j )y_{ij} \leq B_T \\
    \\
    & \forall (i,j) \in E  \\
    & \;\;\;\;\;x_i + x_j - y_{ij} \leq 1 \\
    & \;\;\;\;\;-x_i + y_{ij} \leq 0 \\
    & \;\;\;\;\;-x_j + y_{ij} \leq 0 \\
    & \forall \;i \in O \\
    & \;\;\;\;\;t_{i} \leq d_{i,j}(1-x_j) + M x_j + M x_i\;\; \forall j \in V \\[1pt]\\
    & t_{i} \in \text{Real Number} \\
    & x_i \in \lbrace 0,1 \rbrace \;\; \forall  i \in H\\
    & y_{ij} \in \lbrace 0,1 \rbrace \;\; \forall  (i,j) \in E \\
    \end{array}
$$

### Minimize the Total Impact ( Big M method version )

$$ \begin{array}{rll}
    \text{max} & \sum_{ i \in O} m_i \\
    \\[1pt] 
    \text{s.t.} & \sum_{i \in V} c_ix_i +  \sum_{(i,j) \in E} (w_ix_j + w_jx_i) -\sum_{(i,j) \in E}(p_{(i,j)} + w_i+w_j )y_{ij} \leq B_T \\
    \\
    & \forall (i,j) \in E \uplus C \\
    & \;\;\;\;\;x_i + x_j - y_{ij} \leq 1 \\
    & \;\;\;\;\;-x_i + y_{ij} \leq 0 \\
    & \;\;\;\;\;-x_j + y_{ij} \leq 0 \\
    & \forall i \in O \\
    & \;\;\;\;\;m_{i} \leq \sum_{j\in V}w_{(i,j)}(x_j-1) + M(x_i) \\ 
    \\
    & m_i \in \{-\infty, 0 \} \;\; \forall i \in O \\
    & x_i \in \lbrace 0,1 \rbrace \;\; \forall  i \in H\\
    & y_{ij} \in \lbrace 0,1 \rbrace \;\; \forall  (i,j) \in E  \uplus C \\
    \\
    \end{array}
$$

In [ ]:
Memory / APM ( power management ) / Running Time